In [1]:
using AvailablePotentialEnergyFramework, Statistics, JLD, NCDatasets, BenchmarkTools, ProgressMeter

In [2]:
data_dir = "/global/cscratch1/sd/aramreye/for_postprocessing/largencfiles/"
output_dir = "/global/u2/a/aramreye/RamirezReyes_Yang_2020_SpontaneousCyclogenesis/plotsandanimations/"

"/global/u2/a/aramreye/RamirezReyes_Yang_2020_SpontaneousCyclogenesis/plotsandanimations/"

In [3]:
file_list = ["f1e-4_2km_1000km_homoRad_homoSfc_3d.nc",
"f3e-4_2km_1000km_homoRad_homoSfc_3d.nc",
"f5e-4_2km_1000km_homoRad_homoSfc_3d.nc"]

3-element Array{String,1}:
 "f1e-4_2km_1000km_homoRad_homoSfc_3d.nc"
 "f3e-4_2km_1000km_homoRad_homoSfc_3d.nc"
 "f5e-4_2km_1000km_homoRad_homoSfc_3d.nc"

In [4]:
const R = Float32(Dryair.R)
const cp = Float32(Dryair.cp)

1006.0f0

In [26]:
function N2(exp_name,buf1,buf2,buf3,buf4,ind1,ind2)
    data_dir = "$(ENV["SCRATCH"])/for_postprocessing/largencfiles/"
    c1 :: Float32 = Float32(R/cp)
    z = Float32[50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1178.57142857143 1385.71428571429 1671.42857142857 2035.71428571429 2478.57142857143 3000 3600 4200 4800 5400 6000 6600 7200 7800 8400 9000 9600 10200 10800 11400 12000 12600 13200 13800 14400 15000 15600 16200 16800 17400 18000 18600 19200 19800 20400 21000 21600 22200 22800 23400 24000 24600 25200 25800 26400 27000 27600 28200 28800 29400 30000 30600 31200 31800 32400 33000 33600 34200 34800][1:50];
    ds = Dataset(joinpath(data_dir,exp_name))
    P0 = 1f2ds["p"].var[1:50] :: Array{Float32,1}
    N2 = zeros(Float32,50)
    nchunks = 12
    timesteps_per_chunk = (ind2-ind1 + 1) ÷ nchunks
    #@inbounds @showprogress 1  for chunk in 1:nchunks
    @inbounds @showprogress 1 for timeind in ind1:ind2
        #@show initial_tind = ((chunk - 1) * timesteps_per_chunk) + 1
        #@show final_tind = (chunk * timesteps_per_chunk)
        initial_timeind = timeind
        final_timeind = timeind
        NCDatasets.load!(ds["TABS"].var,buf1,1:512,1:512,1:50,initial_timeind:final_timeind)
        NCDatasets.load!(ds["PP"].var,buf2,1:512,1:512,1:50,initial_timeind:final_timeind)
        NCDatasets.load!(ds["QV"].var,buf3,1:512,1:512,1:50,initial_timeind:final_timeind)       
        @inbounds for ind in CartesianIndices(buf1)
            buf2[ind]  = buf2[ind] + P0[ind[3]]
            buf1[ind]  = buf1[ind] * (one(c1) + 1f-3Float32(epsilon)*buf3[ind])
            buf1[ind]  = buf1[ind] * (P0[1]/buf2[ind])^c1
        end
        mean!(buf4,buf1) #xBar_ThetaV  = 
        N2[:] .+= compute_N2(buf4,z)[:]
    end
    N2 ./ (ind2-ind1)
    close(ds)
end    

N2 (generic function with 1 method)

In [ ]:
varinfo()

In [ ]:
1+1

In [21]:
buf1 = Array{Float32,3}(undef,512,512,50)
buf2 = Array{Float32,3}(undef,512,512,50)
buf3 = Array{Float32,3}(undef,512,512,50)
buf4 = Array{Float32,3}(undef,1,1,50);

In [ ]:
#@btime N2("f5e-4_2km_1000km_homoRad_homoSfc_3d.nc",$buf1,$buf2,$buf3,$buf4,600,1200);
#@code_warntype N2("f5e-4_2km_1000km_homoRad_homoSfc_3d.nc",buf1,buf2,buf3,buf4,600,601);

In [27]:
N2_homoRad_homoSfc = N2("f5e-4_2km_1000km_homoRad_homoSfc_3d.nc",buf1,buf2,buf3,buf4,1,1200)
N2_homoRad_homoSfc_f1 = N2("f1e-4_2km_1000km_homoRad_homoSfc_3d.nc",buf1,buf2,buf3,buf4,1,1200)
N2_homoRad_homoSfc_f3 = N2("f3e-4_2km_1000km_homoRad_homoSfc_3d.nc",buf1,buf2,buf3,buf4,1,1200);

Progress: 100%|█████████████████████████████████████████| Time: 0:35:48
Progress: 100%|█████████████████████████████████████████| Time: 0:40:33
Progress: 100%|█████████████████████████████████████████| Time: 0:48:40


In [ ]:
varinfo()

In [6]:
function get_tropo(buf1,buf2,exp_name,z,ind1,ind2)
    data_dir = "$(ENV["SCRATCH"])/for_postprocessing/largencfiles/"
 	mean_qrad = Dataset(joinpath(data_dir,exp_name)) do ds
        mean_qrad = Array{Float32,1}(undef,80);
        for timeind in ind1:ind2
            NCDatasets.load!(ds["QRAD"].var,buf1,1:512,1:512,1:80,timeind:timeind)
            mean_qrad[:] .+= mean!(buf2,buf1)[:]
        end
        mean_qrad
    end
    z[findmin(abs.(mean_qrad[1:50]))[2]]
end

get_tropo (generic function with 1 method)

In [7]:
z_grd = Float32[50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1178.57142857143 1385.71428571429 1671.42857142857 2035.71428571429 2478.57142857143 3000 3600 4200 4800 5400 6000 6600 7200 7800 8400 9000 9600 10200 10800 11400 12000 12600 13200 13800 14400 15000 15600 16200 16800 17400 18000 18600 19200 19800 20400 21000 21600 22200 22800 23400 24000 24600 25200 25800 26400 27000 27600 28200 28800 29400 30000 30600 31200 31800 32400 33000 33600 34200 34800];

In [8]:
tropo_homoRad_homoSfc = get_tropo(buf1,buf4,"f5e-4_2km_1000km_homoRad_homoSfc_3d.nc",z_grd,1,1200)
tropo_homoRad_homoSfc_f1 = get_tropo(buf1,buf4,"f1e-4_2km_1000km_homoRad_homoSfc_3d.nc",z_grd,1,1200)
tropo_homoRad_homoSfc_f3 = get_tropo(buf1,buf4,"f3e-4_2km_1000km_homoRad_homoSfc_3d.nc",z_grd,1,1200)

LoadError: UndefVarError: buf1 not defined

In [1]:
@show tropo_homoRad_homoSfc
@show tropo_homoRad_homoSfc_f1
@show tropo_homoRad_homoSfc_f3

LoadError: UndefVarError: tropo_homoRad_homoSfc not defined

In [2]:
function rossby_radius_of_deformation(brunt_vaisala,tropopause_height,coriolis_parameter)
    return brunt_vaisala*tropopause_height/coriolis_parameter
end

rossby_radius_of_deformation (generic function with 1 method)

In [3]:
N_homoAll = sqrt(mean(N2_homoRad_homoSfc[25:findfirst(==(tropo_homoRad_homoSfc),z_grd)[2]]))
N_homoAll_f1 = sqrt(mean(N2_homoRad_homoSfc_f1[25:findfirst(==(tropo_homoRad_homoSfc_f1),z_grd)[2]]))
N_homoAll_f3 = sqrt(mean(N2_homoRad_homoSfc_f3[25:findfirst(==(tropo_homoRad_homoSfc_f3),z_grd)[2]]))

LoadError: UndefVarError: tropo_homoRad_homoSfc not defined

In [4]:
@show r0_homoAll = rossby_radius_of_deformation(N_homoAll,tropo_homoRad_homoSfc,4.97e-4)
@show r0_homoAll_f1 = rossby_radius_of_deformation(N_homoAll_f1,tropo_homoRad_homoSfc_f1,1f-4)
@show r0_homoAll_f3 = rossby_radius_of_deformation(N_homoAll_f3,tropo_homoRad_homoSfc_f3,3f-4)

LoadError: UndefVarError: N_homoAll not defined